In [1]:
import sys
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import datetime
from tqdm import tqdm
import GPy
from collections import defaultdict
from pathlib import Path
import seaborn as sns
import scipy.stats as stats
from matplotlib.colors import ListedColormap
import warnings
import time
from itertools import product
from joblib import Parallel, delayed
from operator import itemgetter
import sncosmo
import emcee
import corner
import pickle
from multiprocessing import Pool

mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.autolayout'] = True
# mpl.rcParams['figure.dpi'] = 300
# mpl.rcParams['axes.spines.right'] = False
# mpl.rcParams['axes.spines.top'] = False


plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})


def utkarshGrid():
    plt.minorticks_on()
    plt.grid(color='grey',
             which='minor',
             linestyle=":",
             linewidth='0.1',
             )
    plt.grid(color='black',
             which='major',
             linestyle=":",
             linewidth='0.1',
             )
    
# set random number seed to ensure reproducibility
seed = 3
rstate = np.random.RandomState(seed)

## Implementing MCMC using GP Emulator Model

In [2]:
path = os.path.dirname(os.getcwd())
os.chdir(path)
from Emulator.Classes.AllData import AllData
from Emulator.Classes.LightCurve import utkarshGrid, LightCurve
from Emulator.Classes.GP import GP
from Emulator.Classes.GP2D import GP2D
from Emulator.Classes.GP5D import GP5D
os.chdir(os.getcwd() + "/Emulator")
os.getcwd()

'/Users/utkarsh/PycharmProjects/KilonovaeLightCurves/Emulator'

In [3]:
truth_arr = np.array([0.01, 0.13, 60, 4]) #[mejdyn, mejwind, phi, iobs]
mejdyn_guess, mejwind_guess, phi_guess, iobs_guess = [0.1, 0.1, 20, 9]

In [4]:
def train_fluxes(mejdyn = truth_arr[0], mejwind = truth_arr[1], phi = truth_arr[2], 
                 iobs = truth_arr[3], messages = False):
    curr_wv = np.arange(100, 3600, 10)
    set_skip_factor = None
    gp = GP5D("Classes/reference.csv")
    gp.split = 1
    gp.emulator = "start"
    gp.cross_validation = (mejdyn, mejwind, phi, iobs)
    gp.set_wv_range(curr_wv)
    gp.n_comp = 25
    time_shape = None
    gp.save_pca_components(skip_factor = set_skip_factor)
    gp.setXY_cross_validation(mejdyn, mejwind, phi, iobs, messages = False)
    fitting_kernel = GPy.kern.RBF(input_dim=4, variance = 1, lengthscale=1, ARD = True)
    decay_kernel = GPy.kern.Linear(input_dim=4, ARD = True)
    gp.kernel = fitting_kernel * decay_kernel
    gp.model = GPy.models.GPRegression(gp.X,gp.Y,gp.kernel)
    if messages:
        print(gp.model)
    t0 = time.time()
    if messages:
        print(f"[STATUS] Optimizing...")
    gp.model.optimize(messages = False)
    gp.model_predict(include_like = True, messages = False)
    gp.save_pca_initial_validation()
    gp.delete_folder_files("data/pcaComponentsTrained")
    gp.delete_folder_files("data/pcaComponentsTrainedError")
    return gp

In [43]:
gp = train_fluxes()

In [ ]:
def predict_fluxes(mejdyn, mejwind, phi, iobs, gp = gp, extra_item = None):
    gp.validationX = [mejdyn, mejwind, phi, iobs]
    theta = gp.validationX
    gp.model_predict_cross_validation(include_like = True, messages = False) # Save cross validation
    gp.save_trained_data(errors = False, theta = (mejdyn, mejwind, phi, iobs), extra_item = extra_item)
    y = np.load(f"data/pcaTrained/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
    t = gp._t_helper()
    t_matrix = np.repeat(t, gp.num_wv).reshape(len(t), gp.num_wv)
    x = t_matrix
    os.remove(f"data/pcaComponentsTrained/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
    os.remove(f"data/pcaComponentsTrainedError/mejdyn{mejdyn}_mejwind{mejwind}_phi{phi}_iobs{iobs}.npy")
    return x, y

In [ ]:
x, y = predict_fluxes(mejdyn = truth_arr[0], mejwind = truth_arr[1], 
                      phi = truth_arr[2], iobs = truth_arr[3], extra_item = True)
yerr = 0.2 * y
print(x.shape, y.shape)

In [ ]:
def prior(theta):
    """log-prior as a function of parameters `theta`.""" 
    mejdyn, mejwind, phi, iobs = theta
    lim = 1e6
    if mejdyn > 1 or mejdyn < 0.001:
        return -np.inf
    elif mejwind > 1 or mejwind < 0.001:
        return -np.inf
    elif phi > 91 or phi < 0:
        return -np.inf
    elif iobs > 11 or iobs < 0:
        return -np.inf
    else:
        return 0.0

def loglike(theta, x, y, yerr):
    """log-likelihood as a function of parameters `theta`."""
    
    mejdyn, mejwind, phi, iobs = theta
    x, y_model = predict_fluxes(mejdyn, mejwind, phi, iobs)
    logl = - 0.5 * np.sum(((y - y_model)/yerr)**2)
    return logl

def logpost(theta, x=x, y=y, yerr=yerr):
    """(Negative) log-posterior as a function of parameters `theta`."""
    if not np.isfinite(prior(theta)):
        return -np.inf
    mejdyn, mejwind, phi, iobs = theta  # reassign parameters
    logp = prior(theta)  # prior
    logl = loglike(theta, x=x, y=y, yerr = yerr)  # likelihood
    return (logl + logp)  # posterior

In [ ]:
initial = [mejdyn_guess, mejwind_guess, phi_guess, iobs_guess]
ndim = len(initial)
nwalkers = 10
nburn = 1
niter = 10
t_init = time.time()
p0 = [np.array(initial) + 1 * np.random.randn(ndim) for i in range(nwalkers)]
p0 = np.array(p0, dtype = float)
print(p0.shape)

pool = None
sampler = emcee.EnsembleSampler(nwalkers, ndim, logpost, pool = pool)
t0 = time.time()
print("Started Burn-In")
state = sampler.run_mcmc(p0, nburn, progress=False)
print(f"Burn-In Took: {round(time.time() - t0)}s")
sampler.reset()
state = sampler.run_mcmc(state, niter, progress=True)

gp.delete_folder_files("data/pcaTrained")

In [ ]:
# get final chains
samples = sampler.get_chain()
labs = ['mejdyn', 'mejwind', 'phi', "iobs"]

In [ ]:
# plot posterior
corner.corner(samples.reshape(-1, ndim),  # collect samples into N x 3 array
              bins=10,  # bins for histogram
              show_titles=True, quantiles=[0.16, 0.84],  # show median and uncertainties
              labels=labs,
              truths=truth_arr,  # plot truth
              color='darkviolet', truth_color='black',  # add some colors
              **{'plot_datapoints': False, 'fill_contours': True});  # change some default options

In [ ]:
# plot a few chains
plt.figure(dpi = 300, figsize = (10,8))

plt.subplot(4, 1, 1)
[plt.plot(samples[:, i, 0], alpha=0.5) for i in range(nwalkers)]
plt.xlim([0, niter])
plt.xlabel('Iteration')
plt.ylabel(labs[0])

plt.subplot(4, 1, 2)
[plt.plot(samples[:, i, 1], alpha=0.5) for i in range(nwalkers)]
plt.xlabel('Iteration')
plt.ylabel(labs[1])
plt.xlim([0, niter])

plt.subplot(4, 1, 3)
[plt.plot(samples[:, i, 2], alpha=0.5) for i in range(nwalkers)]
plt.xlim([0, niter])
plt.xlabel('Iteration')
plt.ylabel(labs[2])
plt.tight_layout()

plt.subplot(4, 1, 4)
[plt.plot(samples[:, i, 3], alpha=0.5) for i in range(nwalkers)]
plt.xlim([0, niter])
plt.xlabel('Iteration')
plt.ylabel(labs[3])
plt.tight_layout()

In [ ]:
def next_pow_two(n):
    i = 1
    while i < n:
        i = i << 1
    return i

def auto_window(taus, c):
    m = np.arange(len(taus)) < c * taus
    if np.any(m):
        return np.argmin(m)
    return len(taus) - 1

def autocorr_func_1d(x, norm=True):
    x = np.atleast_1d(x)
    if len(x.shape) != 1:
        raise ValueError("invalid dimensions for 1D autocorrelation function")
    n = next_pow_two(len(x))

    # Compute the FFT and then (from that) the auto-correlation function
    f = np.fft.fft(x - np.mean(x), n=2 * n)
    acf = np.fft.ifft(f * np.conjugate(f))[: len(x)].real
    acf /= 4 * n

    # Optionally normalize
    if norm:
        acf /= acf[0]

    return acf

# Following the suggestion from Goodman & Weare (2010)
def autocorr_gw2010(y, c=5.0):
    f = autocorr_func_1d(np.mean(y, axis=0))
    taus = 2.0 * np.cumsum(f) - 1.0
    window = auto_window(taus, c)
    return taus[window]

def autocorr_new(y, c=5.0):
    f = np.zeros(y.shape[1])
    for yy in y:
        f += autocorr_func_1d(yy)
    f /= len(y)
    taus = 2.0 * np.cumsum(f) - 1.0
    window = auto_window(taus, c)
    return taus[window]

In [ ]:
plt.figure(dpi= 300)
for dimension in range(len(initial)):
    chain = sampler.get_chain()[:, :, dimension].T
    N = np.exp(np.linspace(np.log(10), np.log(chain.shape[1]), 20)).astype(int)

    # Compute the estimators for a few different chain lengths
    gw2010 = np.empty(len(N))
    new = np.empty(len(N))
    for i, n in enumerate(N):
        gw2010[i] = autocorr_gw2010(chain[:, :n])
#         new[i] = autocorr_new(chain[:, :n])

    # Plot the comparisons
    plt.loglog(N, gw2010, "o-", label=f"{labs[dimension]}")
#     plt.loglog(N, new, "o-", label=f"New: {labs[dimension]}")
    
    
ylim = plt.gca().get_ylim()
plt.ylim(ylim)
plt.xlabel("number of samples, $N$")
plt.ylabel(r"$\tau$ estimates")
plt.legend(fontsize=14)
plt.plot(N, N / 50.0, "--k", label=r"$\tau = N/50$")     
plt.title(f"Autocorrelation")
print(f"Walkers:{nwalkers}\nIterations:{niter}\nTotal Runtime:{round((time.time()-t_init)/60, 2)}min")